In [ ]:
# Import libraries
import pandas as pd
from pyregressor.symbolic_regressor import SymbolicRegressor
from pyregressor.search import RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import os

# Load and prepare data
csv_path = 'DO.csv'
if not os.path.exists(csv_path):
    csv_path = '/content/DO.csv'

try:
    df = pd.read_csv(csv_path)

    # Renaming the columns
    df = df.rename(columns={'ndvi': 'x0', 'n-WST': 'x1', 'DO': 'y'})

    # Applying Bound-Safe Dynamic Centering

    # 1. Scaling for x0 (NDVI): Target Range [-0.5, 1.0], Center at 0.1
    u_x0 = df['x0'].mean()
    x0_scaled = (df['x0'].values - u_x0) + 0.1
    x0_scaled = np.clip(x0_scaled, -0.5, 1.0)

    # 2. Scaling for x1 (n-WST): Target Range [0.5, 1.0], Center at 0.75
    u_x1 = df['x1'].mean()
    x1_scaled = (df['x1'].values - u_x1) * 2.0 + 0.75
    x1_scaled = np.clip(x1_scaled, 0.5, 1.0)

    # Combine into X (using column_stack to handle 1D to 2D conversion)
    X = np.column_stack([x0_scaled, x1_scaled])
    y = df['y'].values

    print("Data scaled using Adjusted Bound-Safe Dynamic Centering Strategy:")
    print(" - x0 (NDVI): Centered at 0.1, Bounds [-0.5, 1.0]")
    print(" - x1 (n-WST): Centered at 0.75, Bounds [0.5, 1.0]")
    print(f"Stability check (min x0 + x1 + 0.99): {np.min(X[:,0] + X[:,1]) + 0.99:.4f} (> 0)")
    print(f"x0 range: [{np.min(X[:,0]):.4f}, {np.max(X[:,0]):.4f}]")
    print(f"x1 range: [{np.min(X[:,1]):.4f}, {np.max(X[:,1]):.4f}]")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#Hyperparameters
    param_grid = {
        'population_size': list(range(10, 120)),
        'generations': list(range(10, 250)),
        'stopping_criteria': [1e-5],
        'n_iterations': list(range(10, 50)),
        'parsimony_coefficient': [0.0001, 0.001, 0.005, 0.01],
        'max_size': [30],
        'function_set': [['add', 'sub', 'mul', 'div', 'sqrt', 'log', 'tanh', 'neg', 'inv', 'exp',
                          'max', 'min', 'sin', 'cos', 'tan']]
    }

    regressor = SymbolicRegressor()
    search = RandomSearch(regressor, param_grid=param_grid, n_iter=50, random_state=42, verbose=1)
    search.fit(X_train, y_train)

    best_model = None
    best_mae = np.inf
    results = []

    print("\nAll Symbolic Regression Equations by Complexity:")

    for i, model in enumerate(search.models_):
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        expr = model.__str__()

        results.append({
            'Model_Index': i + 1,
            'Equation': expr,
            'R2_Score': r2,
            'MAE': mae
        })

        print(f"\nModel {i+1} Equation:\n{expr}")
        print(f"  ➤ R²: {r2:.4f}, MAE: {mae:.4f}")

        if mae < best_mae:
            best_mae = mae
            best_model = model

    df_equations = pd.DataFrame(results)
    df_equations.to_csv('all_sr_equations.csv', index=False)
    print("\n📁 All equations saved to: all_sr_equations.csv")

    print("\n\n✅ Best Model (Lowest MAE Selection):")
    print(best_model.__str__())
    print(f"MAE      : {best_mae:.4f}")

    y_pred_final = best_model.predict(X_test)
    df_output = pd.DataFrame(X_test, columns=['x0', 'x1'])
    df_output['Observed_DO'] = y_test
    df_output['Predicted_DO'] = y_pred_final
    df_output.to_csv('predicted_DO.csv', index=False)

    print("\n📁 Final prediction results saved to: predicted_DO.csv")
except FileNotFoundError:
    print("Error: DO.csv not found.")
except Exception as e:
     print(f"An error occurred: {e}")
